In [7]:
!pip install selenium
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from selenium import webdriver  
import pandas as pd
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [9]:
# open the website
driver = webdriver.Edge()

url = "https://zu.fang.com/house-a015277-b02315/"
driver.get(url)

In [10]:
# 之前有分别测试过是否能打开下一页和成功找到element， 测试成功后就把之前的代码都删了整合到循环中了）））

In [ ]:
string_list = []

i = 0

while i<9:    # 清河的租房数据只有9页））
    table = driver.find_element("id", "houselistbody")
    rows = table.find_elements(By.TAG_NAME, "dd")

    data = []
    for row in rows:    # 在每一行中
        cells = row.find_elements(By.TAG_NAME, "p")    # 找到想要爬取的数据
        if cells:
            row_data = [cell.text for cell in cells]
            data.append(row_data)
    df = pd.DataFrame(data)
    string_list.append(df)
    print(i+1)    # 展示爬取进度

    # 检查下一页按钮是否存在
    next_page_elements = driver.find_elements(By.XPATH, "//a[contains(text(), '下一页')]")
    if not next_page_elements:
        print('No more pages')
        break
    
    # 如果存在，打开下一页
    try:
        next_page = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '下一页')]"))
         )
        next_page_href = next_page.get_attribute("href")
        driver.get(next_page_href)
        i += 1
    except NoSuchElementException:
        print("No more pages")
        break
    except ElementClickInterceptedException:
        print("ElementClickInterceptedException: Element is not clickable")
        break
    
    
driver.quit()

0
1
2
3
4
5
6
7
8
No more pages


In [12]:
df_full = pd.concat(string_list, axis = 0, ignore_index = True)
df_full

,0,1,2,3,4,5,6
0,清河小营桥电力宿舍南向精装次卧室出租,合租次卧|3户合租|15㎡|朝南,海淀-清河-电力宿舍,距昌平线清河小营桥站约601米。,,交通便利,1800元/月
1,整租 · 清河 · 永泰东里 · 2室1厅,整租|2室1厅|57㎡|朝南北,海淀-清河-永泰东里,距8号线永泰庄站约592米。,,,6100元/月
2,相寓 清河·世华龙樾一期西区·高楼层·2居室,整租|2室1厅|78㎡|朝南北,海淀-清河-世华龙樾一期,距8号线西小口站约216米。,,,7500元/月
3,整租 · 清河 · 永泰园新地标 · 4室2厅,整租|4室2厅|239㎡|朝南,海淀-清河-永泰园新地标,距昌平线清河小营桥站约395米。,,,16000元/月
4,相寓 清河·海清园·高楼层·3居室·次卧1,合租单间|3户合租|15㎡|朝南,海淀-清河-海清园,距昌平线清河小营桥站约255米。,,,2800元/月
...,...,...,...,...,...,...,...
514,海淀强佑清河新城56.24平方米 2室1厅1卫,整租|2室1厅|56㎡|朝东北,海淀-清河-强佑清河新城,距昌平线清河小营桥站约756米。,个人房源,交通便利随时入住,5600元/月
515,海淀强佑清河新城50.12平方米 1室1厅1卫,整租|1室1厅|50㎡|朝南,海淀-清河-强佑清河新城,距昌平线清河小营桥站约756米。,个人房源,交通便利随时入住,5500元/月
516,合租:主卧海淀清河燕清源60平方米 2室1厅1卫,合租主卧|2户合租|60㎡|朝南,海淀-清河-燕清源,距昌平线清河小营桥站约354米。,个人房源,随时入住,5600元/月
517,合租:主卧海淀清河永泰园新地标129平方米 3室2厅,合租主卧|2户合租|129㎡|朝东南,海淀-清河-永泰园新地标,距昌平线清河小营桥站约395米。,个人房源,家电齐全,2300元/月


In [13]:
# rearrange the data

# 将错位数据（5列中的价格数据）转移到6
df_full[5] = df_full[5].str.extract('(\\d+)')
df_full = df_full.fillna('')
df_full.iloc[:,6] = df_full.iloc[:,6] + df_full.iloc[:,5]
df_full[6] = df_full[6].str.extract('(\\d+)')
df_full[6] = df_full[6] + "元/月"
# 删除不相关的数据
df_full = df_full.drop(columns = [0,4,5])
# 将合并在一列的数据分开  并且获取其中的面积信息
df_split = df_full.iloc[:,0].str.split("|", expand = True)
df_final = pd.concat([df_split, df_full], axis = 1)
# 重命名列
df_final.columns = ['租赁方式', '房型', '面积', '朝向','删除', '地址', '交通',  '房租'  ]
df_final = df_final.drop(columns = ['删除'])
new_columns_order = ['地址', '面积', '房租', '交通', '租赁方式', '房型', '朝向']
df_final = df_final[new_columns_order]

df_final


,地址,面积,房租,交通,租赁方式,房型,朝向
0,海淀-清河-电力宿舍,15㎡,1800元/月,距昌平线清河小营桥站约601米。,合租次卧,3户合租,朝南
1,海淀-清河-永泰东里,57㎡,6100元/月,距8号线永泰庄站约592米。,整租,2室1厅,朝南北
2,海淀-清河-世华龙樾一期,78㎡,7500元/月,距8号线西小口站约216米。,整租,2室1厅,朝南北
3,海淀-清河-永泰园新地标,239㎡,16000元/月,距昌平线清河小营桥站约395米。,整租,4室2厅,朝南
4,海淀-清河-海清园,15㎡,2800元/月,距昌平线清河小营桥站约255米。,合租单间,3户合租,朝南
...,...,...,...,...,...,...,...
514,海淀-清河-强佑清河新城,56㎡,5600元/月,距昌平线清河小营桥站约756米。,整租,2室1厅,朝东北
515,海淀-清河-强佑清河新城,50㎡,5500元/月,距昌平线清河小营桥站约756米。,整租,1室1厅,朝南
516,海淀-清河-燕清源,60㎡,5600元/月,距昌平线清河小营桥站约354米。,合租主卧,2户合租,朝南
517,海淀-清河-永泰园新地标,129㎡,2300元/月,距昌平线清河小营桥站约395米。,合租主卧,2户合租,朝东南


In [14]:

!pip install openpyxl


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# 保存为excel文件
df_final.to_excel('Housing_Rent_Data.xlsx', index = False)